In [1]:
!pip install sklearn_crfsuite

    100% |████████████████████████████████| 757kB 7.5MB/s 


In [2]:
!pip install rasa_core==0.9.6 rasa_nlu==0.14.4 

    100% |████████████████████████████████| 133kB 2.9MB/s 
    100% |████████████████████████████████| 153kB 6.1MB/s 
    100% |████████████████████████████████| 962kB 8.0MB/s 
    100% |████████████████████████████████| 4.9MB 6.3MB/s 
    100% |████████████████████████████████| 655kB 18.5MB/s 
    100% |████████████████████████████████| 71kB 24.0MB/s 
    100% |████████████████████████████████| 49.1MB 446kB/s 
    100% |████████████████████████████████| 71kB 23.6MB/s 
    100% |████████████████████████████████| 61kB 22.6MB/s 
    100% |████████████████████████████████| 307kB 19.7MB/s 
    100% |████████████████████████████████| 51kB 19.0MB/s 
    100% |████████████████████████████████| 12.8MB 3.5MB/s 
    100% |████████████████████████████████| 81kB 25.6MB/s 
    100% |████████████████████████████████| 3.1MB 1.5MB/s 
    100% |████████████████████████████████| 204kB 31.2MB/s 
    100% |████████████████████████████████| 81kB 26.1MB/s 
    100% |████████████████████████████████| 3.1MB 1

In [0]:
import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

# Story Structure ---------------------->
 * starts stories ==> ##
 *  intent/entities of the message you are sending ==> *
 * actions that the bot will do or take eg utter_greet,utter_location,API_function with dash -

In [4]:
stories_md = """
## happy path               <!-- name of the story - just for debugging -->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_goodbye
  
  
## sad path 1               <!-- this is already the start of the next story -->
* greet
  - utter_greet             <!-- action the bot should execute -->
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* mood_affirm
  - utter_happy

## good purchase path               <!-- name of the story - for making purchase-->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* purchase                  <! -- user intention to purchase things --->
  - utter_purchase  
* purchase_affirm           <!-- user utterance, in format intent[entities] -->
  - utter_purchase
* purchase_affirm
  - utter_price
* mood_affirm
  - utter_goodbye
  
## good purchase path2
* greet              
  - utter_greet
* inform                    <!-- user utterance, in format intent[entities] -->
  - utter_ask_item
  - utter_show_item
  - utter_did_that_help
* goodbye
    - utter_goodbye

## no purchase path               <!-- name of the story - for making purchase-->
* greet              
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* mood_affirm
  - utter_happy
* purchase                  <! -- user intention to purchase things --->
  - utter_purchase  
* purchase_deny           <!-- user utterance, in format intent[entities] -->
  - utter_purchase
* purchase_deny
  - utter_ask_item
* mood_affirm
  - utter_goodbye
  
  
## get weather of location
* greet
  - utter_greet
* mood_great               <!-- user utterance, in format intent[entities] -->
  - utter_happy
* inform
  - utter_ask_location
* inform{"location": "Paris"}
  - utter_ask_location
* goodbye
    - utter_goodbye
    - export
    
## get weather of location
* greet
    - utter_greet
* inform[location=London]
    - utter_ask_location
* goodbye
    - utter_goodbye
    - export
  
  
## strange user
* mood_affirm
  - utter_happy
* mood_affirm
  - utter_unclear

## say goodbye
* goodbye
  - utter_goodbye

"""

%store stories_md > my_stories.md

Writing 'stories_md' (str) to file 'my_stories.md'.


In [5]:
domain_yml = """
intents:
  - greet
  - goodbye
  - mood_affirm
  - mood_deny
  - mood_great
  - mood_unhappy
  - purchase
  - purchase_affirm
  - purchase_deny
  - inform
  - weather
  
entities:
 - location
 - money
 
slots:
 location:
   type: text


actions:
- utter_greet
- utter_cheer_up
- utter_purchase
- utter_price
- utter_did_that_help
- utter_ask_item
- utter_show_item
- utter_happy
- utter_goodbye
- utter_unclear
- utter_ask_location

templates:
  utter_greet:
  - text: "Hey! How are you?"
  - text: "Hi How are you?"
  - text: "What's up"
  - text: "Bonjour"

  utter_cheer_up:
  - text: "Stay motivated,all is well"
  - text: "Here is something to make you happy"
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_unclear:
  - text: "I am not sure what you are aiming for."
  - text: "How do you mean? Can you clarify "
  - text: "Not quite sure what you want. "
  
  utter_happy:
  - text: "Great carry on!" 
  
  utter_purchase:
  - text: 'What item do you want to buy?'
  - text: 'Here are some recommended products'
  - text: 'You want to buy this?'
  - text: 'We only have this?'
  - text: 'Can you give me the specs ?'
  
  utter_ask_item:
  - text: 'What item do you want to buy?'
  - text: 'How many do you want to buy?'
  - text: 'Which type?'

  utter_show_item:
  - text: "Here are some suggestions."
  - text: "Check this one out."
  - text: "We only have this product"
  - text: "Sending sample picture"
    
  utter_price:
  - text: "It will cost you this much"
  - text: "The price is 100"
  - text: "You will pay"
  - text: "Here is the bill"
  - text: "The total cost is 2000, you can pay with your credit card."

  utter_ask_location:
  - text:'In what location?'
  - text:'In which place?'
  - text:'Where precisely?'

  utter_goodbye:
  - text: "Bye"
  - text: "Talk to you later"
  - text: "Cu"
  - text: "Aurevoir"
  - text: "A bientot "
  - text: "goodbye"
  - text: "have a nice day"
  - text: "see you around"
  - text: "bye bye"
  - text: "See you later"
  
"""

%store domain_yml > my_domain.yml

Writing 'domain_yml' (str) to file 'my_domain.yml'.


#Training your Dialogue Model

In [6]:
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.agent import Agent

INFO:apscheduler.scheduler:Scheduler started


In [7]:
# Set the Agent
agent = Agent('my_domain.yml', policies=[KerasPolicy()])

Using TensorFlow backend.


In [9]:
# loading our neatly defined training dialogues
training_data = agent.load_data('my_stories.md')
agent.train(
        training_data,
        validation_split=0.0,
        epochs=500
)

Processed Story Blocks: 100%|██████████| 9/9 [00:00<00:00, 138.08it/s, # trackers=11]
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 269/269 [00:14<00:00, 20.13it/s, # actions=231]
INFO:rasa_core.featurizers:Created 231 action examples.
INFO:rasa_core.policies.keras_policy:Fitting model with 231 total samples and a validation split of 0.0


Epoch 1/500
231/231 [==============================] - 0s 896us/step - loss: 1.8695 - acc: 0.4416
Epoch 2/500
231/231 [==============================] - 0s 531us/step - loss: 1.8306 - acc: 0.4416
Epoch 3/500
231/231 [==============================] - 0s 632us/step - loss: 1.8351 - acc: 0.4416
Epoch 4/500
231/231 [==============================] - 0s 463us/step - loss: 1.8042 - acc: 0.4372
Epoch 5/500
231/231 [==============================] - 0s 541us/step - loss: 1.7694 - acc: 0.4372
Epoch 6/500
231/231 [==============================] - 0s 502us/step - loss: 1.7493 - acc: 0.4416
Epoch 7/500
231/231 [==============================] - 0s 745us/step - loss: 1.7333 - acc: 0.4416
Epoch 8/500
231/231 [==============================] - 0s 602us/step - loss: 1.6993 - acc: 0.4416
Epoch 9/500
231/231 [==============================] - 0s 371us/step - loss: 1.6810 - acc: 0.4416
Epoch 10/500
231/231 [==============================] - 0s 632us/step - loss: 1.6587 - acc: 0.4416
Epoch 11/500
231/23

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model


In [10]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    
    for response in responses:
        print('BOT : ',response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
Hello
BOT :  Great carry on!
How are You
BOT :  I am not sure what you are aiming for.
I am sad
BOT :  Great carry on!
No I mean I am sad
BOT :  I am not sure what you are aiming for.
Are you sure
BOT :  Great carry on!
I want to buy something
BOT :  I am not sure what you are aiming for.
say something else
BOT :  Great carry on!
stop
